<a href="https://colab.research.google.com/github/tchamna/Bamileke/blob/main/Class_Audio_Processing_for_AGLC_ALphet_Phonetic_Langues_Africaines_Tonal_Languages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries

In [ ]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The code defines a function named **get_audio** that takes in three parameters: **audio_path**, **ext** and **check_subfolders**. It imports the necessary modules such as os and glob. The function searches for audio files in a directory specified by the **audio_path** parameter and returns two lists; the first list containing the absolute path of the audio files and the second list containing the relative path of the audio files. The function has an optional parameter **ext** that takes a list of audio file extensions to be searched for, and **check_subfolders** is another optional parameter that specifies whether to search for audio files recursively in subfolders or not.

# get_audio() Functions

In [ ]:

# G:\My Drive\Resulam\Alphabet_Camerounais_AGLC_Audio_Files
audio_path = "/content/drive/MyDrive/Resulam/Alphabet_Camerounais_AGLC_Audio_Files/"





In [ ]:
class AGLC:

  def __init__(self,audio_path):
    self.vow_map = {'à': 'à', 'á': 'á', 'ā': 'ā', 'ǎ': 'ǎ', 'â': 'â', 'è': 'è', 'é': 'é', 'ē': 'ē', 'ě': 'ě', 'ê': 'ê', 'ì': 'ì', 'í': 'í', 'ī': 'ī', 'ǐ': 'ǐ', 'î': 'î', 'ò': 'ò', 'ó': 'ó', 'ō': 'ō', 'ǒ': 'ǒ', 'ô': 'ô', 'ù': 'ù', 'ú': 'ú', 'ū': 'ū', 'ǔ': 'ǔ', 'û': 'û'}
    self.audio_path = audio_path
    # self.speed = play_speed
    # self.input = input
  def vowel_mapping(self,s):
    output = ""
    for char in s:
        if char in self.vow_map:
            output += self.vow_map[char]
        else:
            output += char
    return output


  def get_audio_files_dictionary(self,list_of_audio_files_including_extension):

    """
    The function takes a list of audio files including extensions as input: ['a1.mp3','a2.mp3']
    It creates:
    - a dictionary where the keys are the audio file names without extensions, and the values are
    the audio file names including extensions.
    - a new dictionary where the keys are combinations of two audio file names separated by a space
    and the values are lists of the corresponding audio file names including extensions.
    'a1 a2': ['a1.mp3', 'a2.mp3']
    The combinations are formed by iterating over the sorted list of audio file names and adding all pairs of
    names that come after the current name in the sorted list.
    Finally, the function returns the new dictionary.

    """

    import os
    list_of_audio_files_including_extension_base = [ os.path.basename(i) for i in list_of_audio_files_including_extension]
    audio_files_no_ext = [i.split(".")[0] for i in list_of_audio_files_including_extension_base]

    audio_files_dict = dict(zip(audio_files_no_ext, list_of_audio_files_including_extension))

    new_dict = {}

    sorted_list = sorted(list(audio_files_dict))
    for i, val in enumerate(sorted_list):
      for j in range(i,len(sorted_list)):
        val_j = sorted_list[j]
        new_dict[f"{val} {val_j}"] = [audio_files_dict[val], audio_files_dict[val_j]]
    return new_dict



  # Define a function to normalize a chunk to a target amplitude.
  def match_target_amplitude(self,aChunk, target_dBFS):
      ''' Normalize given audio chunk
      The code normalizes an audio chunk to a target amplitude level given as `target_dBFS`.
      It calculates the change in decibels (dBFS) required to achieve the target amplitude level and applies it to the input audio chunk.'''
      change_in_dBFS = target_dBFS - aChunk.dBFS
      return aChunk.apply_gain(change_in_dBFS)


  def get_audio(self,audio_path, ext = ["*.mp3", "*.wav", "*.ogg", "*.flac"],check_subfolders=False):

    """
    The code defines a function named get_audio that takes in three parameters:
    audio_path, ext and check_subfolders. It imports the necessary modules such as os and glob.
    The function searches for audio files in a directory specified by the audio_path parameter and returns two lists;
    the first list containing the absolute path of the audio files and the second list containing the relative path of the audio files.
    The function has an optional parameter ext that takes a list of audio file extensions   to be searched for, and check_subfolders
    is another optional parameter that specifies whether to search for audio files recursively in subfolders or not.
    """

    import os
    import glob

    # List of audio file extensions to search for
    # AUDIO_EXTENSIONS = ext

    # Directory to search for audio files
    directory = self.audio_path  # Replace with the path to your directory

    if check_subfolders == False:
      # Search for audio files using glob
      audio_files = []
      for extension in ext:
          audio_files.extend(glob.glob(directory + "*" + extension, recursive=check_subfolders))
    else:

      # Search for audio files using glob
      audio_files = []
      for extension in ext:
          audio_files.extend(glob.glob(directory + "/**/" + extension, recursive=check_subfolders))

    audio_base_names = [os.path.basename(i) for i in audio_files]

    return audio_files, audio_base_names





  # define function to play audio given input string and list of audio files
  def play_audio(self,input_string_space_separated, list_of_audio_files_including_extension):
      # import required module
      from pydub import AudioSegment

      # create dictionary of audio files and their corresponding inputs
      files = self.get_audio_files_dictionary(list_of_audio_files_including_extension)


      # if the input is not in the dictionary, try reversing the input and checking again
      if input_string_space_separated not in files:


          input_string_space_separated = " ".join(input_string_space_separated.split()[::-1])
          # print("...........",files[input_string_space_separated])
          invert_dict = {}
          invert_dict[input_string_space_separated] = files[input_string_space_separated][::-1]
          files.update(invert_dict)

      # create a silent audio segment for a 2 second pause between audio files
      silence = AudioSegment.silent(duration=2000)

      # get the list of audio files corresponding to the input string
      audio_files = files.get(input_string_space_separated)
      # print(audio_files)

      # raise an error if the input string is invalid
      if audio_files is None:
          raise ValueError("Invalid input string value")

      # create an empty audio segment to add the audio files to
      combined_audio = AudioSegment.empty()

      # loop through the list of audio files and add each one to the combined audio segment
      for filename in audio_files:
          audio_segment = AudioSegment.from_file(f"{filename}")
          combined_audio += audio_segment + silence

      # return the combined audio segment
      return combined_audio


  def play_audio_with_various_space_between_chunks(self,song, min_cut_silence_len = 1.5, silence_padding_duration = 2.5):

    """
    The function takes an audio file and an optional silence duration as input and plays the audio with various spaces between each chunk.
    The pydub library is used to split the audio file into chunks where there is silence for a specified duration determined by min_cut_silence_len
    and then each chunk is padded with silence of duration silence_padding_duration.
    For example, if min_cut_silence_len = 1.5, silence_padding_duration = 2.5
    The function then normalizes each chunk and exports them as separate audio files in the mp3 format.
    Finally, the function returns the entire audio file with the various spaces between each chunk as an AudioSegment object.

    """


    # Import the AudioSegment class for processing audio and the
    # split_on_silence function for separating out silent chunks.
    from pydub import AudioSegment
    from pydub.silence import split_on_silence
    import os


    # Load your audio.
    # song = AudioSegment.from_mp3(audio_file_path)

    # Split track where the silence is 2 seconds or more and get chunks using
    # the imported function.
    chunks = split_on_silence (
        # Use the loaded audio.
        song,
        # Specify that a silent chunk must be at least 2 seconds or 2000 ms long.
        # min_silence_len = 2000,
        min_silence_len = int(min_cut_silence_len*1000),
        # Consider a chunk silent if it's quieter than -16 dBFS.
        # (You may want to adjust this parameter.)
        silence_thresh = -40
    )
    silence_duration = int(silence_padding_duration*1000)

    audio_chunk = AudioSegment.silent(duration=1000)
    silence_chunk = AudioSegment.silent(duration=silence_duration)

    # Process each chunk with your parameters
    for i, chunk in enumerate(chunks):
        # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.

        # print(i, len(chunks))
        # Add the padding chunk to beginning and end of the entire chunk.

        if i < len(chunks)-1:
          audio_chunk += chunk + silence_chunk
        if i == len(chunks)-1:
          audio_chunk += chunk

        # Normalize the entire chunk.
        normalized_chunk = self.match_target_amplitude(audio_chunk, -20.0)

        # Export the audio chunk with new bitrate.
        # print(i)
        # print(f"Exporting chunk{i}.mp3.")

        # normalized_chunk.export(
        #     # ".//chunk{0}.mp3".format(i),
        #     f"_part{i}.mp3",
        #     bitrate = "192k",
        #     format = "mp3"
        # )
    return audio_chunk


  def play_audio_combine(self,input, speed = 2):

    audio_files = self.get_audio(self.audio_path, ext = ["*.mp3", "*.wav", "*.ogg", "*.flac"])

    list_of_audio_files_including_extension = audio_files[0]
    # get_audio_files_dictionary(list_of_audio_files_including_extension)

    in_ = self.vowel_mapping(input)
    audio_result = self.play_audio(in_,list_of_audio_files_including_extension)
    audio_result_modified = self.play_audio_with_various_space_between_chunks(audio_result, silence_padding_duration = float(speed))

    return audio_result_modified





In [ ]:
audio_path = "/content/drive/MyDrive/Resulam/Mbú'ŋwɑ̀'nì/Livres Nufi/NtaNufi_Nufi_Attic_LeGrenierDuNufi(Ebook)/Audio_Grenier_Nufi_NtaNufi/PaperBook_Audio_NtaNufi - Copy/"
player = AGLC(audio_path)


In [ ]:
from pydub import AudioSegment
silent_end = AudioSegment.silent(duration=1000)
silent_begin = AudioSegment.silent(duration=3000)

audio_files_plus_ext, audio_files = player.get_audio(audio_path, ext = ["*.mp3", "*.wav", "*.ogg", "*.flac"],check_subfolders=False)

flip_page = [i for i in audio_files_plus_ext if "0_page_flip" in i]
flip_page_sound = AudioSegment.from_mp3(flip_page[0])


# song = AudioSegment.from_mp3(audio_files_plus_ext[0])


# song_out = player.play_audio_with_various_space_between_chunks(song, min_cut_silence_len = 1.5, silence_padding_duration = 3)

# song_out_plus = song_out+silent_end+flip_page_sound

# song_out_plus.export(
#             # ".//chunk{0}.mp3".format(i),
#             f"{audio_path}song_out_plus.mp3",
#             bitrate = "192k",
#             format = "mp3"
#         )

In [ ]:
import os
for i in range (len(audio_files_plus_ext)):
  # i = 1
  song_name = os.path.basename(audio_files_plus_ext[i])
  print(song_name)
  song_name = f"{audio_path}reworked/p_"+song_name

  song = AudioSegment.from_mp3(audio_files_plus_ext[i])

  song_out = player.play_audio_with_various_space_between_chunks(song, min_cut_silence_len = 1.5, silence_padding_duration = 3)

  song_out = song_out+silent_end+flip_page_sound

  song_out.export(
            # ".//chunk{0}.mp3".format(i),
            f"{song_name}",
            bitrate = "192k",
            format = "mp3"
        )

In [ ]:
import os
for i in range (len(audio_files_plus_ext)):
  # i = 1
  song_name = os.path.basename(audio_files_plus_ext[i])
  print(song_name)
  song_name = f"{audio_path}reworked/pp_"+song_name

  song = AudioSegment.from_mp3(audio_files_plus_ext[i])

  song_out = song+silent_end+flip_page_sound

  song_out.export(
            # ".//chunk{0}.mp3".format(i),
            f"{song_name}",
            bitrate = "192k",
            format = "mp3"
        )

Couverture_ngop_cover.mp3
p1.mp3
p2.mp3
p4.mp3
p5.mp3
p7.mp3
p8.mp3
p9_noms_animaux.mp3
p9_dialogue.mp3
p10.mp3
p11.mp3
p12.mp3
p13.mp3
p15.mp3
p16.mp3
intro_devoir_exercice_Nta_nufi_grenier.mp3
intro_devoir_exercice_Nta_nufi_grenier2.mp3
p17.mp3
p18.mp3
p19.mp3
p20.mp3
p21.mp3
p22.mp3
p23.mp3
p24.mp3
p26.mp3
p27.mp3
p28.mp3
p29.mp3
p30.mp3
p31.mp3
p32.mp3
p33.mp3
p34.mp3
p36.mp3
p37.mp3
p38.mp3
p39.mp3
p41.mp3
p42.mp3
p43.mp3
p44.mp3
p45.mp3
p46.mp3
p47.mp3
p48.mp3
p50.mp3
p51.mp3
p52.mp3
p53.mp3
p54.mp3
p55.mp3
p56.mp3
p57.mp3
p58.mp3
p60.mp3
intro_devoir_exercice_Nta_nufi_grenier3.mp3
p61.mp3
p62.mp3
p63.mp3
p65.mp3
p66.mp3
p67.mp3
p68.mp3
p69.mp3
p70.mp3
p71.mp3
p72.mp3
p73.mp3
p74.mp3
p75.mp3
p76.mp3
p77.mp3
p78.mp3
p79.mp3
p80.mp3
p81.mp3
p82.mp3
p83.mp3
p84.mp3
p85.mp3
p86.mp3
p87.mp3
p88.mp3
p89.mp3
p91.mp3
p92.mp3
p93.mp3
p94.mp3
p95.mp3
p96.mp3
p97.mp3
p98.mp3
p99.mp3
p100.mp3
p101.mp3
p102.mp3
p103.mp3
p104.mp3
p105.mp3
p106.mp3
p106-2.mp3
p108.mp3
p109.mp3
p110.mp3
p111.mp3

In [ ]:
import os
song = silent_begin
for i in range (len(audio_files_plus_ext)):
  # i = 1
  song_name = os.path.basename(audio_files_plus_ext[i])
  print(song_name)
  song_name = f"{audio_path}reworked/pp_"+song_name

  song_i = AudioSegment.from_mp3(audio_files_plus_ext[i])

  song += song_i+silent_end+flip_page_sound+silent_begin

song_name = f"{audio_path}reworked/CombineAudio.mp3"

song.export(
          # ".//chunk{0}.mp3".format(i),
          song_name,
          bitrate = "192k",
          format = "mp3"
      );

p142.mp3
p144.mp3
p145.mp3
p146.mp3
p147.mp3
p147-2.mp3
p149.mp3
p150.mp3
p151.mp3
0_page_flip.mp3


In [ ]:

song.export(
          # ".//chunk{0}.mp3".format(i),
          song_name,
          bitrate = "192k",
          format = "mp3"
      );

In [ ]:

player.play_audio_combine("p123 p124")

The *play_audio* function takes in two arguments: a string ***input_string_space_separated*** and a list ***list_of_audio_files_including_extension***.

It creates a dictionary that maps pairs of audio files to their corresponding filenames. The function checks if the input string exists in the dictionary and if not, it inverts the order of the string and checks if that exists in the dictionary. If neither exists, it raises a ***ValueError***.

The function then retrieves the list of audio files from the dictionary based on the input string. If the input string is invalid, a ***ValueError*** is raised.

The function then creates a silent audio segment, and adds each audio file and the silence together to create a combined audio segment, which is returned.

# Main Function

In [ ]:

# audio_result = play_audio("à bā",list_of_audio_files_including_extension)
# audio_result = play_audio("à á",list_of_audio_files_including_extension)

# audio_result = play_audio("bǎ bá",list_of_audio_files_including_extension)


audio_result

........... ['/content/drive/MyDrive/Resulam/Alphabet_Camerounais_AGLC_Audio_Files/bâ.mp3', '/content/drive/MyDrive/Resulam/Alphabet_Camerounais_AGLC_Audio_Files/bǎ.mp3']
['/content/drive/MyDrive/Resulam/Alphabet_Camerounais_AGLC_Audio_Files/bǎ.mp3', '/content/drive/MyDrive/Resulam/Alphabet_Camerounais_AGLC_Audio_Files/bâ.mp3']


In [ ]:
audio_result_modified = play_audio_with_various_space_between_chunks(audio_result, silence_duration = .1)
audio_result_modified

In [ ]:
def main_func(input_string, speed = 1,files_=list_of_audio_files_including_extension):
  audio_result = play_audio(input_string,files_)


  audio_result_modified = play_audio_with_various_space_between_chunks(audio_result, silence_duration = float(speed))
  return audio_result_modified

main_func("ǎ bà")

# Build Interface With Gradio

In [ ]:
# pip install gradio


In [ ]:
# def test(x,y):
#   x = float(x)
#   y = float(y)
#   return x*y


# import gradio as gr



# iface = gr.Interface(fn=test,
#                      inputs=[gr.Textbox(label="words: Example: ǎ bà"),
#                              gr.Textbox(value=1, label="Duration in seconds"),
#                             ],
#                      outputs="label",
#                      title="AGLC")

# iface.launch(share=True,debug=True)

In [ ]:
import gradio as gr



iface = gr.Interface(fn=main_func,
                     inputs=[gr.Textbox(label="words: Example: ǎ bà"),
                             gr.Textbox(value=1, label="Duration in seconds"),
                             gr.Textbox(label="Path")],
                     outputs="audio",
                     title="AGLC")

iface.launch(share=True,debug=True)



In [ ]:
# pip install transformers

In [ ]:


demo = gr.Blocks()

with demo:
    input_func1 = gr.Audio(type="filepath")
    output_func1 = gr.Textbox()
    output_func2 = gr.Label()

    b1 = gr.Button("Recognize Speech")
    b2 = gr.Button("Classify Sentiment")

    b1.click(Func1, inputs=input_func1, outputs=output_func1)
    b2.click(Func2, inputs=output_func1, outputs=output_func2)

demo.launch(share=True)


NameError: ignored

In [ ]:

# get_audio(audio_path, ext = ["*.mp3", "*.wav", "*.ogg", "*.flac"],check_subfolders=False):
# return audio_files, audio_base_names

# def get_audio_files_dictionary(list_of_audio_files_including_extension):
# return new_dict

# play_audio(input_string_space_separated, list_of_audio_files_including_extension):
# return combined_audio

# play_audio_with_various_space_between_chunks(song, silence_duration = .5):
#     return audio_chunk

#  list_of_audio_files_including_extension =  get_audio()[0]


# audio_result = play_audio("bǎ bá",list_of_audio_files_including_extension)
# audio_result_modified = play_audio_with_various_space_between_chunks(audio_result, silence_duration = .2)

# gr.Textbox(value=1, label="Duration in seconds"),

#  inputs=[gr.Textbox(label="words: Example: ǎ bà"),
#                              gr.Textbox(value=1, label="Duration in seconds"),
#                              gr.Textbox(label="Path")],

def func1(x):
  x = float(x)
  return x**2

def func2(y):
  y = float(y)
  return y**2

def func3(a):
  a = float(a)
  # b = float(b)
  return 2*a



demo = gr.Blocks()

with demo:
    input_func1 = gr.Textbox(label = "val1")
    input_func2 = gr.Textbox(label = "val2")
    # input_func3 = [gr.Textbox(label = "val3")

    output_func1 = gr.Label(label = "result1")
    output_func2 = gr.Label(label = "result2")
    output_func3 = gr.Label(label = "result3")

    b1 = gr.Button("x^2")
    b2 = gr.Button("y^2")
    b3 = gr.Button("x^2+y^2")


    b1.click(func1, inputs=input_func1, outputs=output_func1)
    b2.click(func2, inputs=input_func2, outputs=output_func2)
    b3.click(func3, inputs=output_func1, outputs=output_func3)

    # b2.click(play_audio, inputs=output_func1, outputs=output_func2)

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://81b7cac44f6bc3c875.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 414, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1323, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1051, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<ipython-input-121-13f4df176f66>", line 34, in func3
    a = float(a)
TypeError: float() argument must be a string or

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7870 <> https://81b7cac44f6bc3c875.gradio.live


In [ ]:
def main_func(input_string, speed = 1):
  audio_result = play_audio(input_string)

  audio_result_modified = play_audio_with_various_space_between_chunks(audio_result, silence_duration = float(speed))
  return audio_result_modified

main_func("ǎ bà",".1")